In [1]:
print("Ok")

Ok


In [9]:
%pwd

'c:\\Users\\LENOVO\\Desktop\\VS code programs\\HealthCare\\AI_Medical_Chatbot\\research'

In [10]:
import os
os.chdir("../")

In [11]:
%pwd

'c:\\Users\\LENOVO\\Desktop\\VS code programs\\HealthCare\\AI_Medical_Chatbot'

In [12]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader (data,
                         glob="*.pdf",
                         loader_cls=PyPDFLoader)

    documents= loader.load()

    return documents

In [15]:
extracted_data= load_pdf_file(data='Data/')

In [17]:
# extracted_data